# QAP VQE

In [ ]:
#INITIALISATION CELL
import qiskit
from qiskit import Aer
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.circuit.library import RealAmplitudes,TwoLocal #(uses CX entangling) 
from qiskit.aqua import QuantumInstance

from docplex.mp.model import Model
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.applications.ising import docplex

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

# useful additional packages 
%matplotlib inline
import numpy as np
import pandas as pd
import math

In [ ]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [ ]:
#small sized matrices(under 10x10) (quick on all methods)
matrix_size_3 = 'made3.csv'
matrix_size_4 = 'made4.csv'
matrix_size_5 = 'made5.csv'
matrix_size_6 = 'made6.csv'
matrix_size_7 = 'made7.csv'
matrix_size_8 = 'made8.csv'
matrix_size_9 = 'made9.csv'

In [ ]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """
    Input:
    z (list[int]): list of allocations
    MatrixLoc (numpy array): matrix of distances
    MatrixFlow (numpy array): matrix of flow
    
    Output:
    float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length)).astype(int)
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

In [ ]:
from qiskit import IBMQ

# Load account from disk
IBMQ.load_account()
IBMQ.providers()

In [ ]:
#Insert your provider details if necessary
provider = IBMQ.get_provider()
provider.backends()

In [ ]:
def qap_feasible(x):
    """
    Input:
    x (numpy.ndarray) : binary string as numpy array.

    Ouput:
    bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n)).astype(int)
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [ ]:
def testing_quantum(machine, ins, SPtrial, circ,  SHOT, optimal_point):
    """
    Input:
    machine: the name of available machine
    ins: the QAP instance
    SPtrial: max_trials for SPSA
    circ: 'RA' or 'TL' 
    SHOT: number of shots
    optimal_point: numpy array of optimal points from simulator
    
    Output:
    outmatrix: 30 * [eigensolution value, time, feasibility]
    (outfile "thirty_trials-<machine name>-<instance name>.csv")
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] 
                       for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) 

    #Setup VQE
    seed = 10598
    spsa = SPSA(maxiter=SPtrial)
    print('num of qubits; ', qubitOp_docplex.num_qubits)
    if circ == 'RA':
        ry = RealAmplitudes(qubitOp_docplex.num_qubits, entanglement='linear')
    if circ == 'TL':
        ry = TwoLocal(qubitOp_docplex.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
    
    #30 trials
    file = open("thirty_trials-" + str(machine) + "-" + str(ins) ,"w")
    file.write("value,   feasible,   frequency, time, iteration" + "\n")
    ans = np.zeros(5)
    for i in range(30):
        try:
            #VQE
            vqe = VQE(qubitOp_docplex, ry, spsa, include_custom=True, initial_point = optimal_point )
            #backend = Aer.get_backend(machine)
            backend = provider.get_backend(machine)
            quantum_instance = QuantumInstance(backend=backend,seed_simulator=seed, seed_transpiler=seed, 
                                               skip_qobj_validation = False, shots = SHOT)
            result = vqe.run(quantum_instance,)

            #Output processing
            print(' Eigenstate:', result['eigenstate'])
            print('VQE time:', result['optimizer_time'])
            n = len(list(result['eigenstate'].values()))
            solution = np.hstack((np.array(list(result['eigenstate'].values())).reshape(n,1),
                                  np.array(list(result['eigenstate'].keys())).reshape(n,1)))
            print(solution)
            for r in solution:
                file.write(str(qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc)) + "," + 
                           str(qap_feasible(np.array(list(r[1])))) + "," + str(r[0]) + ","+ 
                           str(result['optimizer_time']) + "," + str(i) + "\n")
                ans = np.vstack((ans,np.array([qap_value(np.array(list(r[1])),MatrixFlow,MatrixLoc),
                                               qap_feasible(np.array(list(r[1]))),float(r[0]),result['optimizer_time'],i])))
        except:
            file.write("An error occurred on iteration " + str(i) + " of the 30 trials")
            print("AN ERROR OCCURRED ON ITERATION" + str(i) + " of the 30 trials")
            ans = np.vstack((ans,np.array([math.inf,False,0,np.nan,i])))
        print("Iteration "+ str(i) + " is complete.")
    file.close()
    return ans[1:] 

In [ ]:
def read_optimal(ins):
    data = pd.read_csv("initial_point-" + str(ins),header = None)
    n = len(data[0]) - 1
    ans = []
    data[0][0] = data[0][0][1:]
    data[0][n] = data[0][n][:-1]
    for i in data[0]:
        r = i.split(" ")
        for t in r:
            if t!='':
                ans.append(float(t))
    return np.array(ans)

## The main cells

These are the cells running solutions. It should take between 2 hours and 2 days per cell. Note errors are printed but the code should continue

In [108]:
#3 by 3 thirty trials

import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
machines = 'ibmq_cambridge'
#machines = 'qasm_simulator'
instances = "made3.csv"
optimal_point = read_optimal(instances)
ans3 = testing_quantum(machines, instances, 1, "TL", 1024,optimal_point)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x_

INFO:qiskit.transpiler.runningpassmanager:Pass: UnrollCustomDefinitions - 0.00000 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('cz', 2), ('ry', 1)} to target basis {'z', 'u1', 'y', 'cu3', 'cswap', 'rz', 'tdg', 'u2', 'cx', 'rx', 'cy', 't', 'ccx', 'crz', 'barrier', 'rxx', 'cu1', 'snapshot', 'cry', 'measure', 'ryy', 'crx', 'swap', 'h', 'ry', 'x', 'sdg', 's', 'u3', 'ch', 'id', 'cz', 'reset'}.
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Begining basis search from frozenset({('cz', 2), ('ry', 1)}) to frozenset({'z', 'u1', 'cu3', 'rz', 'tdg', 'u2', 'rx', 'cy', 'crz', 'snapshot', 'cry', 'crx', 'swap', 'h', 'ry', 'x', 'sdg', 's', 'u3', 'id', 'y', 'cswap', 'cx', 't', 'ccx', 'barrier', 'rxx', 'cu1', 'measure', 'ryy', 'ch', 'cz', 'reset'}).
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Transformation path:
INFO:qiskit.transpiler.passes.basis.basis_translator:Basis translation path search completed in 0.004s.
INFO:qiskit.trans

INFO:qiskit.aqua.quantum_instance:
Qiskit Terra version: 0.15.1
Backend: 'qasm_simulator (AerProvider)', with following setting:
{'basis_gates': ['u1', 'u2', 'u3', 'cx', 'cz', 'id', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg', 'swap', 'ccx', 'unitary', 'diagonal', 'initialize', 'cu1', 'cu2', 'cu3', 'cswap', 'mcx', 'mcy', 'mcz', 'mcu1', 'mcu2', 'mcu3', 'mcswap', 'multiplexer', 'kraus', 'roerror'], 'coupling_map': None}
{'pass_manager': None, 'initial_layout': None, 'seed_transpiler': 10598, 'optimization_level': None}
RunConfig(max_credits=10, seed_simulator=10598, shots=1024)
{'timeout': None}
{}
{}
Measurement mitigation: None


num of qubits;  16


INFO:qiskit.aqua.algorithms.vq_algorithm:Starting optimizer.
bounds=[(None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, 

...

INFO:qiskit.aqua.quantum_instance:
Qiskit Terra version: 0.15.1
Backend: 'qasm_simulator (AerProvider)', with following setting:
{'basis_gates': ['u1', 'u2', 'u3', 'cx', 'cz', 'id', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg', 'swap', 'ccx', 'unitary', 'diagonal', 'initialize', 'cu1', 'cu2', 'cu3', 'cswap', 'mcx', 'mcy', 'mcz', 'mcu1', 'mcu2', 'mcu3', 'mcswap', 'multiplexer', 'kraus', 'roerror'], 'coupling_map': None}
{'pass_manager': None, 'initial_layout': None, 'seed_transpiler': 10598, 'optimization_level': None}
RunConfig(max_credits=10, seed_simulator=10598, shots=1024)
{'timeout': None}
{}
{}
Measurement mitigation: None
INFO:qiskit.aqua.algorithms.vq_algorithm:Starting optimizer.
bounds=[(None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (N

DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updated transform for mapped instr ('cz', 2) cz, [] to
                                      
q2162_0: ──────────────■──────────────
         ┌──────────┐┌─┴─┐┌──────────┐
q2162_1: ┤ U2(0,pi) ├┤ X ├┤ U2(0,pi) ├
         └──────────┘└───┘└──────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Composing transform step: cx/2 [] =>
global phase: 3pi/4
     ┌───┐     ┌────────┐┌───┐     ┌────────┐┌───┐┌───┐     
q_0: ┤ H ├─────┤0       ├┤ X ├─────┤0       ├┤ H ├┤ S ├─────
     ├───┤┌───┐│  ISWAP │├───┤┌───┐│  ISWAP │├───┤├───┤┌───┐
q_1: ┤ X ├┤ H ├┤1       ├┤ X ├┤ H ├┤1       ├┤ S ├┤ X ├┤ H ├
     └───┘└───┘└────────┘└───┘└───┘└────────┘└───┘└───┘└───┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updating transform for mapped instr ('cz', 2) cz, [] from 
                                      
q2162_0: ──────────────■──────────────
         ┌──────────┐┌─┴─┐┌──────────┐
q2162_1: ┤ U2(0,pi) ├┤ X ├┤ U2(0,pi) ├
         └─

INFO:qiskit.transpiler.runningpassmanager:Pass: UnrollCustomDefinitions - 1.99986 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('barrier', 16), ('cz', 2), ('ry', 1), ('measure', 1)} to target basis {'mcu2', 'z', 'u1', 'cu3', 'u2', 'tdg', 'mcx', 'roerror', 'mcu3', 'snapshot', 'mcz', 'mcy', 'mcu1', 'swap', 'h', 'x', 'sdg', 's', 'u3', 'id', 'unitary', 'y', 'cswap', 'kraus', 'cx', 't', 'ccx', 'initialize', 'barrier', 'cu1', 'measure', 'multiplexer', 'cu2', 'mcswap', 'diagonal', 'cz', 'reset'}.
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Begining basis search from frozenset({('barrier', 16), ('cz', 2), ('ry', 1), ('measure', 1)}) to frozenset({'mcu2', 'z', 'u1', 'cu3', 'u2', 'tdg', 'mcx', 'roerror', 'mcu3', 'snapshot', 'mcz', 'mcy', 'mcu1', 'swap', 'h', 'x', 'sdg', 's', 'u3', 'id', 'unitary', 'y', 'cswap', 'kraus', 'cx', 't', 'ccx', 'initialize', 'barrier', 'cu1', 'measure', 'multiplexer', 'cu2', 'mcswap', 'diagonal', 'cz', 're

DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updating transform for mapped instr ('ry', 1) ry, ['ry[0]'] from 
         ┌───────────┐
q2168_0: ┤ ry(ry[0]) ├
         └───────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updated transform for mapped instr ('ry', 1) ry, ['ry[0]'] to
         ┌───────────────┐
q2168_0: ┤ R(ry[0],pi/2) ├
         └───────────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Composing transform step: r/1 [Parameter(theta), Parameter(phi)] =>
     ┌───────────────────────────────────────────────────────────┐
q_0: ┤ U3(theta,phi - 1.5707963267949,1.5707963267949 - 1.0*phi) ├
     └───────────────────────────────────────────────────────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updating transform for mapped instr ('ry', 1) ry, ['ry[0]'] from 
         ┌───────────────┐
q2168_0: ┤ R(ry[0],pi/2) ├
         └───────────────┘
DEBUG:qiskit.transpiler.passes.basis.basis_translator:Updated transform for mapped instr (

 Eigenstate: {'0001000101000100': 1, '0001000101001000': 1, '0000000101001000': 2, '0010000101001000': 1007, '0010000101011000': 1, '0010000110001000': 1, '0010001001001000': 1, '0010001010001000': 2, '0010001011001000': 1, '0010001101001000': 2, '0010011001001000': 2, '0010100101001000': 1, '0101100101001000': 1, '1101000101001000': 1}
VQE time: 1.155038833618164
[['1' '0001000101000100']
 ['1' '0001000101001000']
 ['2' '0000000101001000']
 ['1007' '0010000101001000']
 ['1' '0010000101011000']
 ['1' '0010000110001000']
 ['1' '0010001001001000']
 ['2' '0010001010001000']
 ['1' '0010001011001000']
 ['2' '0010001101001000']
 ['2' '0010011001001000']
 ['1' '0010100101001000']
 ['1' '0101100101001000']
 ['1' '1101000101001000']]
Iteration 29 is complete.


In [ ]:
#4 by 4 thirty trials

import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
machines = 'ibmq_cambridge'
#machines = 'qasm_simulator'
instances = "made4.csv"
optimal_point = read_optimal(instances)
ans4 = testing_quantum(machines, instances, 1, "TL", 1024,optimal_point)

In [ ]:
#5 by 5 thirty trials

import logging
logging.basicConfig(level=logging.DEBUG) # log the steps of the algorithm and results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
machines = 'ibmq_cambridge'
#machines = 'qasm_simulator'
instances = "made5.csv"
optimal_point = read_optimal(instances)
ans3 = testing_quantum(machines, instances, 1, "TL", 1024,optimal_point)

In [5]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.15.1',
 'qiskit-aer': '0.6.1',
 'qiskit-ignis': '0.4.0',
 'qiskit-ibmq-provider': '0.8.0',
 'qiskit-aqua': '0.7.5',
 'qiskit': '0.20.0'}